# 1 Prepear Work

In [1]:
import os.path, jieba

In [2]:
import pandas as pd

In [3]:
word_content_path = r'D:\Assignment\Project_01\news-word-cut.txt'

In [4]:
related_word_path = r'D:\Assignment\Project_01\related_word.txt'

In [5]:
def generate_word_list(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    article_contents_list = []
    i = 0
    while content_line:
        temp = ''
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            for word in content_line:
                if word == ' ':
                    article_contents_list += [temp]
                    temp = ''
                else:
                    temp = temp + word
        content_line = file.readline()
        if temp != '': article_contents_list += [temp]
    return article_contents_list

In [6]:
word_content = generate_word_list(word_content_path)

In [7]:
word_content[0:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']

In [8]:
import gensim

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
word2vec = gensim.models.word2vec.Word2Vec.load("./Word2vec_model.w2v").wv

In [10]:
word2vec.most_similar('体验版', topn=10)

[('稳定版', 0.9151314496994019),
 ('乐东黎族自治县', 0.9127017259597778),
 ('御史大夫', 0.91154944896698),
 ('范丽青', 0.9077432155609131),
 ('饰件', 0.9041963815689087),
 ('陈瑞谢', 0.9034069776535034),
 ('酥碱', 0.9024229049682617),
 ('北京火车站', 0.902195155620575),
 ('长缆', 0.9020404815673828),
 ('诺氟沙星', 0.8997544050216675)]

In [11]:
def get_related_word(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    result = []
    i = 0
    while content_line:
        temp = ''
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            result.append(content_line)
        content_line = file.readline()

    return result

In [12]:
related_word = get_related_word(related_word_path)

In [13]:
related_word[:10]

['说', '指出', '表示', '认为', '坦言', '透露', '看来', '告诉', '提到', '所说']

# 2 SIF

## 2.1 Get the probality of word

In [14]:
from collections import Counter

In [15]:
unigrams_dict = Counter(word_content)

word_list_len = len(word_content)

def get_unigrams_probability(word):
    esp = 1 / word_list_len
    if word in unigrams_dict:
        return unigrams_dict[word] / word_list_len 
    else:
        return esp

In [16]:
get_unigrams_probability('啊')

4.5933924963048614e-05

## 2.2 Sentences embdding

In [17]:
import numpy as np

In [18]:
from sklearn.decomposition import PCA

In [19]:
def get_SIF(word, a=0.01):
    return (a + get_unigrams_probability(word))

In [20]:
def SIF_sentence_embdding(sentence, a=0.01):
    result = 0
    i = 0
    for word in sentence:
        temp = word2vec[word] * get_SIF(word)
        result += temp
    return result

In [21]:
def PCA_SIF_sentence_embdding(news, embedding_size=35, a=0.01):
    X = [SIF_sentence_embdding(sentence) for sentence in news]
    
    pca = PCA(n_components=min(embedding_size, len(X)))
    
    pca.fit(np.array(X))
    
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT
    
    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below
    # resulting sentence vectors, vs = vs -u x uT x vs
    Y = []
    for Vs in X:
        sub = np.multiply(u, Vs)
        Y.append(np.subtract(Vs, sub))
    return Y

In [22]:
from collections import defaultdict

In [23]:
def generate_all_PCA_SIF_sentence_embdding(content):
    result = []
    for news in content:
        yield PCA_SIF_sentence_embdding(news)

In [24]:
#sentence_embdding = generate_all_PCA_SIF_sentence_embdding(sentence_word_content)

In [25]:
from scipy.spatial.distance import cosine

In [26]:
def distance(v1, v2):
    return cosine(v1, v2)

# 3 NER & Dependency Prasing

In [27]:
from stanfordcorenlp import StanfordCoreNLP

In [28]:
nlp = StanfordCoreNLP(r'D:\stanford_nlp', lang='zh')

# 4 execute program

In [29]:
import re, jieba

In [30]:
def token(string):
    return ''.join(re.findall(r'[\d|\w]+', string))
def cut(string):
    return ' '.join(jieba.cut(string))
def cut_sentence(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [31]:
def get_sentence_content_with_punctuation(news):
    news = re.sub(r'\n', '', news)
    if news != '' and news != ' ':
        result = cut_sentence(news)
    return result

In [32]:
def get_sentence_content(news_sentence_with_punctuation):
    result = []
    
    for sentence in news_sentence_with_punctuation:
        temp = token(sentence)
        result.append(temp)
    return result

In [33]:
def get_sentence_word_content(news_sentence):
    result = []
    
    for sentence in news_sentence:
        temp_word = jieba.lcut(sentence)
        result.append(temp_word)
    return result

In [37]:
def get_person_and_point(input_text):
    PERSON = []
    V_point = defaultdict(list)
    flag = -1
    
    news_sentence_with_punctuation = get_sentence_content_with_punctuation(input_text)
    news_sentence = get_sentence_content(news_sentence_with_punctuation)
    
    news_word_with_punctuation = get_sentence_word_content(news_sentence_with_punctuation)
    news_word = get_sentence_word_content(news_sentence)
    
    for j, sentence in enumerate(news_word_with_punctuation):
        for i, word in enumerate(sentence):
            if word in related_word:
                flag = i
                break

        if flag >= 0:
            Y = PCA_SIF_sentence_embdding(news_word)
            
            temp_person = ''
            temp_sen = news_sentence_with_punctuation[j]
            temp_point = [temp_sen]
            
            NER = nlp.ner(temp_sen)
            D_pr = nlp.dependency_parse(temp_sen)
            
            for i, relation in enumerate(D_pr):
                if relation[1] == flag+1 and relation[0] == 'nsubj' and (NER[relation[-1] - 1][-1] == 'PERSON' or NER[relation[-1] - 1][-1] == 'ORGANIZATION'):
                    temp_person = sentence[relation[-1] - 1]
                    PERSON.append(sentence[relation[-1] - 1])
                    for m in range((len(news_word_with_punctuation) - j -1)):
                        m += (j+1)
                        if(distance(Y[j], Y[m])) < 0.2:
                            temp_point.append(news_sentence_with_punctuation[j])
                        else:
                            break
            if temp_person != '':
                V_point[temp_person] = temp_point
        flag = -1
    return PERSON, V_point

In [35]:
test = '（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交警发现有一女子赤裸上身，行走在南坪快速上，期间还起了轻生年头，一辅警发现后赶紧为其披上黄衣，并一路劝说她。\n那么事发时\n到底都发生了些什么呢？\n南都记者带您一起还原现场\n南都记者在龙岗大队坂田中队见到了辅警刘青（发现女生的辅警），一位外表高大帅气，说话略带些腼腆的90后青年。\n刘青介绍，6月16日早上7时36分，他正在环城南路附近值勤，接到中队关于一位女子裸身进入机动车可能有危险的警情，随后骑着小铁骑开始沿路寻找，大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子。\n女子身上一丝不挂地逆车流而行，时走时停，时坐时躺，险象环生。刘青停好小铁骑，和另外一名巡防员追了上去，发现女子的情绪很低落，话不多，刘青尝试和女子交流，劝说女子离开，可女子并不愿意接受，继续缓慢地往南坪快速路的主干道上走去。\n此时路边上已经聚集了很市民围观，为了不刺激女子的情绪，刘青和巡防员一边盯着女子一边驱赶着围观的群众。\n现场还原\n从警方提供的一份视频了解到，16日早上7时25分，女子出现在坂雪岗大道与环城南路的监控视频中，此时女子还穿着白色的内裤，正沿着坂雪岗大道往南坪快速的方向缓慢地走着。\n当时正值上班高峰期，十字路口的车流已经排起了长队。当女子出现时，路上的市民纷纷驻足观望，不少车辆也放慢了速度，但女子并不为市民观望停下脚步，依然缓慢走着。当女子行进到十字路口中间时，一辆大货车挡住了镜头，但是当女子再次出现镜头时，可以发现女子已经没穿内裤了，全身裸露继续朝着南坪快速方向走去。记者发现，视频中女子周围并没有人尾随或者上前劝止的市民。\n一大清早路上看到这样的情况\n恐怕大家都没办法淡定\n面对这一情况\n刘青表示，“一开始根本不敢看她，心里挺别扭，感觉很尴尬”，但当刘青跟随女子上了南坪快速路主干道时，女子作出了让人意想不到的举动，她突然靠近护栏要从上面跳下去，刘青赶忙冲上去拉住了女子的手，将其控制住并远离护栏。碍于女子没有穿衣服，刘青递上衣服，女子没接受还把衣服扔到排水沟里，继续往前走，没办法刘青只能紧紧拉着她的一只手跟在后面。\n刘青一路上耐心地开导安慰她，但只听到她不断地重复着一句话“要是你也遭遇我的事，你也会这样的”，期间她还不时试图挣脱刘青的手要冲向护栏往下跳。\n就这样，我被牵着走了大概十多分钟，天突然下起了大暴雨，雨大的连眼睛都睁不开”刘青继续说着，瞬间他们就被雨透了，但女子依然不愿意接受刘青的帮助，就继续冒着大雨往前走。\n大概走了有四十分钟吧，女子突然停下来说“我想回家了”，然后女子也接受了刘青递过来的小黄衣，就出现了深圳微博上的照片，女子披着小黄衣，刘青小心翼翼地在旁边走着的场景。从南平快速下来后，刘青和巡防员将女子带到了附近的坂田派出所。\n那姑娘到底是遭遇了什么样的事情\n才会说\n“要是你也遭遇我的事，你也会这样”\n据警方透露，该女子姓陈，系湖北人，今年44岁，据家属反映其有精神病史。三天前，陈某从老家来深圳约会网友，但约会受挫导致情绪异常，女子遂产生轻生念头。\n目前\n陈某已经被送往深圳某精神病医院进行治疗\n大大君只希望姑娘能早点康复\n其实真爱的到来并不存在年龄的限制\n你们说呢？\n因善良的原因\n一众网友纷纷为\n交警暖男点ZAN\n@弓常yan桦：就想问这个小哥哥有女票吗\n@原谅我这一辈子浪荡不羁爱萨摩耶：有什么过不去的要轻生嘛？ 想想自己的家人。同时也感谢交警蜀黍\n@火心聆听心灵：点赞交警\n@中華云盾：警察……警察就是群众最需时申出援手\n@Tomchlee：蜀黍帅！\n@SJ-李赫海i：这个交警很暖有木有！\n男子迷奸网友拍418个视频 女方从20岁到50岁不等\n去年6月7号上午，淮安市涟水县公安局刑警大队突然接到了一个奇怪的报警电话，一名女子言语不清，声称自己遭到了侵害。女子、被侵害、言语不清，几个关键词令接到电话的民警瞬间紧张起来。\n'


In [38]:
PERSON, V_point = get_person_and_point(test)

In [39]:
PERSON

['刘青']

In [40]:
V_point

defaultdict(list,
            {'刘青': ['刘青介绍，6月16日早上7时36分，他正在环城南路附近值勤，接到中队关于一位女子裸身进入机动车可能有危险的警情，随后骑着小铁骑开始沿路寻找，大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子。']})